In [19]:
import os
import pandas as pd
import random
import torch
import SimpleITK as sitk
import numpy as np
import torchvision
from torchsummary import summary

In [3]:
tumor_patients = os.listdir('F:/all_users/shendc/workspace/data/CT/ai/data')
no_tumor_patients = os.listdir('F:/all_users/shendc/workspace/data/CT/zhengchang/data')

random.shuffle(tumor_patients)
random.shuffle(no_tumor_patients)

In [10]:
test_sets = {'file_path': ['F:/all_users/shendc/workspace/data/CT/zhengchang/data/'+ each for each in no_tumor_patients[-15:]] + ['F:/all_users/shendc/workspace/data/CT/ai/data/'+ each for each in tumor_patients[-15:]],
             'name':no_tumor_patients[-15:] + tumor_patients[-15:],
             'label':[0]*len(no_tumor_patients[-15:]) + [1]*len(tumor_patients[-15:])}
valid_sets = {'file_path': ['F:/all_users/shendc/workspace/data/CT/zhengchang/data/'+ each for each in no_tumor_patients[-30:-15]] + ['F:/all_users/shendc/workspace/data/CT/ai/data/'+ each for each in tumor_patients[-30:-15]],
             'name':no_tumor_patients[-30:-15] + tumor_patients[-30:-15],
             'label':[0]*len(no_tumor_patients[-30:-15]) + [1]*len(tumor_patients[-30:-15])}
train_sets = {'file_path': ['F:/all_users/shendc/workspace/data/CT/zhengchang/data/'+ each for each in no_tumor_patients[:-30]] + ['F:/all_users/shendc/workspace/data/CT/ai/data/'+ each for each in tumor_patients[:-30]],
             'name':no_tumor_patients[:-30] + tumor_patients[:-30],
             'label':[0]*len(no_tumor_patients[:-30]) + [1]*len(tumor_patients[:-30])}

In [18]:
df = pd.DataFrame(train_sets)
df.to_csv('F:/all_users/shendc/workspace/data/CT/train_set.csv')
df = pd.DataFrame(valid_sets)
df.to_csv('F:/all_users/shendc/workspace/data/CT/valid_set.csv')
df = pd.DataFrame(test_sets)
df.to_csv('F:/all_users/shendc/workspace/data/CT/test_set.csv')

In [19]:
aug = random.sample(['x_flip', 'y_flip'], k=1)[0]

In [15]:
patients = os.listdir('../data/CT/zhengchang/data') + os.listdir('../data/CT/ai/data')
clinical_patients = pd.read_csv('../data/linchuang.csv')
clinical_patients = [each + '.nii.gz' for each in clinical_patients['name'].values]
ra_patients = pd.read_csv('../data/fangshe.csv')
ra_patients = list(ra_patients['fileName'].values)
zong = pd.read_csv('../data/zong.csv')
zong = list(zong['fileName'].values)


In [34]:
clinical = pd.read_csv('../data/linchuang.csv')
clinical

,name,INR,APTT,Fib,TT,CA199,CA125,label
0,Cai Jin Hua,1.14,28.3,4.62,13.3,15.10,15.10,0
1,Cai Xiao Di,0.96,29.4,3.17,13.8,8.01,9.08,0
2,Cai Zu Chang,1.06,30.1,2.67,16.8,9.63,14.90,0
3,Cao Ding Ping,1.09,29.6,2.05,16.9,17.59,9.34,0
4,Cao Feng Man,1.14,27.7,6.03,13.0,62.80,12.35,0
...,...,...,...,...,...,...,...,...
293,zhudayuan,1.15,25.2,2.41,18.8,315.80,13.58,1
294,zhuhuide,1.00,35.5,3.22,14.4,11.12,13.50,1
295,zhupeifang,0.98,25.8,2.79,17.1,10.20,16.70,1
296,zhuyoujun,1.07,24.9,1.77,19.3,98.03,105.20,1


In [35]:
data = clinical.iloc[::, 1:-1]
data

,INR,APTT,Fib,TT,CA199,CA125
0,1.14,28.3,4.62,13.3,15.10,15.10
1,0.96,29.4,3.17,13.8,8.01,9.08
2,1.06,30.1,2.67,16.8,9.63,14.90
3,1.09,29.6,2.05,16.9,17.59,9.34
4,1.14,27.7,6.03,13.0,62.80,12.35
...,...,...,...,...,...,...
293,1.15,25.2,2.41,18.8,315.80,13.58
294,1.00,35.5,3.22,14.4,11.12,13.50
295,0.98,25.8,2.79,17.1,10.20,16.70
296,1.07,24.9,1.77,19.3,98.03,105.20


In [36]:
max_num = data.max()
min_num = data.min()

# 求归一化
data_normal = (data - min_num) / (max_num - min_num)
clinical.iloc[::, 1:-1] = data_normal
name1 = (clinical[clinical['label'] == 0]['name'] + '.nii.gz').values
name2 = (clinical[clinical['label'] == 1]['name'] + '.nii').values
name = np.concatenate([name1, name2])

In [37]:
clinical['name'] = name

In [41]:
clinical.to_csv('../data/linchuang_normalized.csv')

In [17]:
clinical = pd.read_csv('../data/linchuang_normalized.csv').values[::, 2:4]
df = pd.DataFrame(clinical, columns=['0', '1'])
clinical = df.values

torch.empty(1, 197, 768)

tensor([[[9.6805e-39, 6.6005e-07, 1.0489e-08,  ..., 6.5635e-07,
          6.8256e-07, 1.6900e-04],
         [6.5643e-07, 5.0715e-39, 2.5903e-09,  ..., 4.6104e-39,
          1.7440e+28, 4.1203e+24],
         [1.9603e-19, 1.1259e+24, 1.7638e+25,  ..., 6.6014e-07,
          1.0780e-08, 4.2189e-08],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]])

In [55]:
benign_other_patients = os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\张伟俊\ÕÅÎ°¿¡')+\
                        os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\333 新华 良性疾病 吴倩芸')+\
                        os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\新华 良性病变 吴倩芸\1')
benign_xirou_patients = os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\颜伟康\胆囊息肉')
benign_rouyazhong_patients = os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\颜伟康\黄色肉芽肿')
benign_xianjizheng_patients = os.listdir(r'F:\all_users\gaohy\data\gall bladder\xinhua\BenigntumorXinhua\颜伟康\腺肌症')

count_dict = {'胆囊息肉':0, '黄色肉芽肿':0, '腺肌症':0, '其余':0, '正常胆囊':0}
benign_patients = [each.split('.')[0] for each in os.listdir(r'F:\all_users\shendc\workspace\data\CT\zhengchang\data')]

In [56]:
for each in benign_patients:
    if each in benign_other_patients:
        count_dict['其余'] += 1
    elif each in benign_xirou_patients:
        count_dict['胆囊息肉'] += 1
    elif each in benign_rouyazhong_patients:
        count_dict['黄色肉芽肿'] += 1
    elif each in benign_xianjizheng_patients:
        count_dict['腺肌症'] += 1
    else:
       count_dict['正常胆囊'] += 1 

count_dict

{'胆囊息肉': 6, '黄色肉芽肿': 4, '腺肌症': 10, '其余': 52, '正常胆囊': 76}